In [1]:
import os
import yaml

config = yaml.safe_load(open("myconfig.yml"))

os.environ["OPENAI_API_KEY"] = config["OPENAI_API_KEY"]
os.environ["TAVILY_API_KEY"] = config["TAVILY_API_KEY"]
os.environ["LANGCHAIN_API_KEY"] = config["LANGCHAIN_API_KEY"]
os.environ["LANGCHAIN_HUB_API_KEY"] = config["LANGCHAIN_API_KEY"]
os.environ["LANGCHAIN_TRACING_V2"] = str(config["LANGCHAIN_TRACING_V2"]).lower()
os.environ["LANGCHAIN_ENDPOINT"] = config["LANGCHAIN_ENDPOINT"]
os.environ["LANGCHAIN_HUB_API_URL"] = config["LANGCHAIN_HUB_API_URL"]
os.environ["LANGCHAIN_WANDB_TRACING"] = str(config["LANGCHAIN_WANDB_TRACING"]).lower()
os.environ["WANDB_PROJECT"] = config["WANDB_PROJECT"]

# Tools

![./images/Screenshot 2024-02-21 at 20.49.04.png](<./images/Screenshot 2024-02-21 at 20.49.04.png>)

**First** such tool-augmented architecture started in 2022 with A21Labs 

These modules can be neural (e.g. deep learning models) or symbolic (e.g. math calculator, currency converter, weather API).

ChatGPT [Plugins](https://openai.com/blog/chatgpt-plugins) and **OpenAI API** [function calling](https://platform.openai.com/docs/guides/gpt/function-calling) are good examples of LLMs augmented with tool use capability working in practice

### Memory as a retrieval tool

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool

In [3]:
loader = WebBaseLoader("https://neurons-lab.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    "neurons_lab_search",
    "Search for information about Neurons Lab. For any questions about Neurons Lab, you must use this tool!",
)

### Langchain tools

More tools here https://python.langchain.com/docs/integrations/tools

LangChain offers various agent types, each suited for specific use cases. Here are some of the available agents:

- Zero-shot ReAct: This agent uses the ReAct framework to choose tools based solely on their descriptions. It requires descriptions for each tool and is highly versatile.
- Structured input ReAct: This agent handles multi-input tools and is suitable for complex tasks like navigating a web browser. It uses a tools' argument schema for structured input.
- OpenAI Functions: Specifically designed for models fine-tuned for function calling, this agent is compatible with models like gpt-3.5-turbo-0613 and gpt-4-0613. We used this to create our first agent above.
- Conversational: Designed for conversational settings, this agent uses ReAct for tool selection and utilizes memory to remember previous interactions.
- Self-ask with search: This agent relies on a single tool, "Intermediate Answer," which looks up factual answers to questions. It's equivalent to the original self-ask with search paper.
- ReAct document store: This agent interacts with a document store using the ReAct framework. It requires "Search" and "Lookup" tools and is similar to the original ReAct paper's Wikipedia example.

You can read more about these agents in [this blog article](https://nanonets.com/blog/langchain/)

In [4]:
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.tools.tavily_search import TavilySearchResults
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI

In [5]:
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)

In [6]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)
agent_chain = initialize_agent(
    [retriever_tool, tavily_tool],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

/home/p72admin/miniconda3/envs/autogpt/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/home/p72admin/miniconda3/envs/autogpt/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [14]:
# run the agent
agent_chain.run(
    "What are the Neurons Lab services"
)



> Entering new AgentExecutor chain...
I should use the neurons_lab_search tool to find information about the services offered by Neurons Lab.
Action: neurons_lab_search
Action Input: Neurons Lab services
Observation: Neurons Lab | Home


























 



Your vision
Our expertise
Secret sauce*


                            Recipe                            



Partner with us


                                for INNOVATION                                                    
We are AI solution development experts for fast-growing companies seeking innovation through collaboration.






















HealthTech
CleanTech
RetailTech





AWS Certified Advanced partner
and Generative AI competency holder






AI2 = Accelerators2 + 
Handbook2


AI feasibility analysis
Discover


AI solution engineering
Launch


AI operations management
Scale





Success =(Execution x Talent) / 
(Uncertainty x Time)

All  stories

                                    1 — 6                 

'The Neurons Lab services include AI solution development, AI feasibility analysis, AI solution engineering, AI operations management, and Creative Practice Solutions for HealthTech.'

### Custom tools and reasoning

Here is a full guide on how to create [custom tools for LangChain](https://python.langchain.com/docs/modules/agents/tools/custom_tools)

In [8]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

In [10]:
@tool
def calculate_length_tool(a: str) -> int:
    """The function calculates the length of the input string."""
    return len(a)

In [11]:
calculate_length_tool.description, calculate_length_tool.args,

('calculate_length_tool(a: str) -> int - The function calculates the length of the input string.',
 {'a': {'title': 'A', 'type': 'string'}})

In [12]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)
agent_chain = initialize_agent(
    [retriever_tool, tavily_tool, calculate_length_tool],
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [13]:
# run the agent
agent_chain.run(
    "Find the description of Neurons Lab services and calculate the length of this description",
)



> Entering new AgentExecutor chain...
I should use the neurons_lab_search tool to find the description of Neurons Lab services and then use the calculate_length_tool to calculate the length of the description.
Action: neurons_lab_search
Action Input: "Neurons Lab services description"
Observation: Neurons Lab | Home


























 



Your vision
Our expertise
Secret sauce*


                            Recipe                            



Partner with us


                                for INNOVATION                                                    
We are AI solution development experts for fast-growing companies seeking innovation through collaboration.






















HealthTech
CleanTech
RetailTech





AWS Certified Advanced partner
and Generative AI competency holder






AI2 = Accelerators2 + 
Handbook2


AI feasibility analysis
Discover


AI solution engineering
Launch


AI operations management
Scale





Success =(Execution x Talent) / 
(Uncertainty

'The length of the description of Neurons Lab services is 107.'